## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2. Data Preprocessing

There are two main datasets (Dataset 1 and Dataset 2) for each season that will be used to extract the features needed for the model. 

First we create an empty DataFrame. This DataFrame will eventually contain the data integrated from the two datasets.

In [3]:
df = pd.DataFrame()

In [4]:
seasons = ['2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', 
           '2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

In [5]:
# standardize the teams names across all datasets
rename_teams = {}

with open('team_names.json') as json_file:
    rename_teams = json.load(json_file)

## 2.1 Dataset 1

Data Source: www.football-data-co.uk

Firstly we will concatenate every Dataset 1 for every season into a single DataFrame. Then we will do some data preprocessing steps on the DataFrame.

In [6]:
dataset1_df = pd.DataFrame()

for season in seasons:
    # read csv file for match statistics
    temp_df = pd.read_csv(f'../data/processed/data-source-1/{season}-data-source-1.csv')
    
    # remove rows with no HomeTeam information
    temp_df = temp_df[temp_df['HomeTeam'].notna()]
    
    # rename team names in the 'HomeTeam' and 'AwayTeam' columns for standardized team names
    temp_df['HomeTeam'] = temp_df['HomeTeam'].apply(lambda word : rename_teams[word])
    temp_df['AwayTeam'] = temp_df['AwayTeam'].apply(lambda word : rename_teams[word])
    
    # concatenate temp_df to dataset1_df
    if dataset1_df.empty:
        dataset1_df = temp_df
    else:
        dataset1_df = pd.concat([dataset1_df, temp_df]).reset_index(drop=True)

In [7]:
# Make sure we have 5 seasons x 380 matches = 1900 matches in the DataFrame
dataset1_df.shape

(3800, 26)

In [8]:
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Time
0,18/08/12,arsenal,sunderland,0,0,D,0,0,D,C Foy,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.50,8.50,NaN
1,18/08/12,fulham,norwich,5,0,H,2,0,H,M Oliver,11,4,9,2,12,11,6,3,0,0,0,0,1.80,3.60,4.50,NaN
2,18/08/12,newcastle,tottenham,2,1,H,0,0,D,M Atkinson,6,12,4,6,12,8,3,5,2,2,0,0,2.50,3.40,2.75,NaN
3,18/08/12,qpr,swansea,0,5,A,0,1,A,L Probert,20,12,11,8,11,14,5,3,2,2,0,0,2.00,3.40,3.80,NaN
4,18/08/12,reading,stoke,1,1,D,0,1,A,K Friend,9,6,3,3,9,14,4,3,2,4,0,1,2.38,3.25,3.10,NaN


### 2.1.1 Converting Data Types

In [9]:
dataset1_df.dtypes

Date         object
HomeTeam     object
AwayTeam     object
FTHG          int64
FTAG          int64
FTR          object
HTHG          int64
HTAG          int64
HTR          object
Referee      object
HS            int64
AS            int64
HST           int64
AST           int64
HF            int64
AF            int64
HC            int64
AC            int64
HY            int64
AY            int64
HR            int64
AR            int64
B365H       float64
B365D       float64
B365A       float64
Time         object
dtype: object

In [10]:
def standardizeDate(date):
    year = date.split('/')[-1]
    
    if len(year) == 2:
        date = pd.to_datetime(date, format="%d/%m/%y")
    elif len(year) == 4:
        date = pd.to_datetime(date, format="%d/%m/%Y")
        
    return date

# convert 'Date' column to datetime object
dataset1_df['Date'] = dataset1_df['Date'].apply(lambda x: standardizeDate(x))
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Time
0,2012-08-18,arsenal,sunderland,0,0,D,0,0,D,C Foy,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.50,8.50,NaN
1,2012-08-18,fulham,norwich,5,0,H,2,0,H,M Oliver,11,4,9,2,12,11,6,3,0,0,0,0,1.80,3.60,4.50,NaN
2,2012-08-18,newcastle,tottenham,2,1,H,0,0,D,M Atkinson,6,12,4,6,12,8,3,5,2,2,0,0,2.50,3.40,2.75,NaN
3,2012-08-18,qpr,swansea,0,5,A,0,1,A,L Probert,20,12,11,8,11,14,5,3,2,2,0,0,2.00,3.40,3.80,NaN
4,2012-08-18,reading,stoke,1,1,D,0,1,A,K Friend,9,6,3,3,9,14,4,3,2,4,0,1,2.38,3.25,3.10,NaN


### 2.1.2 Checking for Missing Values

In [11]:
dataset1_df.isnull().sum()

Date           0
HomeTeam       0
AwayTeam       0
FTHG           0
FTAG           0
FTR            0
HTHG           0
HTAG           0
HTR            0
Referee        0
HS             0
AS             0
HST            0
AST            0
HF             0
AF             0
HC             0
AC             0
HY             0
AY             0
HR             0
AR             0
B365H          0
B365D          0
B365A          0
Time        2660
dtype: int64

In [12]:
# drop Time column
dataset1_df.drop(['Time'], axis=1, inplace=True)

### 2.1.3 Data Transformation

In [13]:
dataset1_df['FTR'].unique()

array(['D', 'H', 'A'], dtype=object)

In [14]:
def transform_FTR(word):
    if word == 'H':
        return 'W'
    elif word == 'A':
        return 'L'
    elif word == 'D':
        return 'D'

dataset1_df['FTR'] = dataset1_df['FTR'].apply(lambda x: transform_FTR(x))
dataset1_df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,2012-08-18,arsenal,sunderland,0,0,D,0,0,D,C Foy,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.50,8.50
1,2012-08-18,fulham,norwich,5,0,W,2,0,H,M Oliver,11,4,9,2,12,11,6,3,0,0,0,0,1.80,3.60,4.50
2,2012-08-18,newcastle,tottenham,2,1,W,0,0,D,M Atkinson,6,12,4,6,12,8,3,5,2,2,0,0,2.50,3.40,2.75
3,2012-08-18,qpr,swansea,0,5,L,0,1,A,L Probert,20,12,11,8,11,14,5,3,2,2,0,0,2.00,3.40,3.80
4,2012-08-18,reading,stoke,1,1,D,0,1,A,K Friend,9,6,3,3,9,14,4,3,2,4,0,1,2.38,3.25,3.10


## 2.2 Dataset 2

Data Source: www.fbref.com/en

Firstly we will concatenate every Dataset 2 for every season into a single DataFrame. Then we will do some data preprocessing steps on the DataFrame.

### 2.2.1 Feature Engineering

While doing data preprocessing for Dataset 2, we will also create two new features **HDays** and **ADays** along the way.

The **HDays** feature indicates the number of days since the home team's last match.

The **ADays** feature indicates the number of days since the away team's last match.

Supposedly, we should create new features in the Feature Engineering step later. However, these two features must be created using information obtained from the raw datasets. It is not possible to create these two features after we have concatenated every Dataset 2 of each season into a single dataframe. Therefore, we will create these two features before we concatenate every Dataset 2 of each season into a single dataframe.

In [15]:
# returns the number of days since home team's last match
def getHDays(row):
    HDaysLastPlayed = str(row['DaysLastPlayed']).split()[0]
    return HDaysLastPlayed

In [16]:
# returns the number of days since away team's last match
def getADays(row):
    
    date = row['Date']
    team = row['team']
    opponent = row['Opponent']
    
    filter_condition = (concatenated_df['Date'] == date) & (concatenated_df['team'] == opponent) & (concatenated_df['Opponent'] == team)
    ADaysLastPlayed = str(concatenated_df[filter_condition]['DaysLastPlayed']).split()[1]    
    return ADaysLastPlayed

In [17]:
dataset2_df = pd.DataFrame()

for season in seasons:
    
    concatenated_df = pd.read_csv(f'../data/processed/data-source-2/{season}-data-source-2.csv')
  
     # convert 'Date' column to datetime object
    concatenated_df['Date'] =  pd.to_datetime(concatenated_df['Date'], format="%Y/%m/%d")
    
    # get DaysLastPlayed for all matches
    concatenated_df['DaysLastPlayed'] = concatenated_df['Date'] - concatenated_df['Date'].shift(1)
    
    # filter by Premier League matches only
    concatenated_df = concatenated_df[concatenated_df['Comp'] == 'Premier League']
    
    # rename team names in the 'Opponent' column for standardized team names
    concatenated_df['Opponent'] = concatenated_df['Opponent'].apply(lambda word : rename_teams[word])
    
    # add a new feature: HDays (number of days since home team's last match)
    concatenated_df['HDays'] = concatenated_df.apply(lambda row: getHDays(row), axis=1)
    
    # add a new feature: ADays (number of days since away team's last match)
    concatenated_df['ADays'] = concatenated_df.apply(lambda row: getADays(row), axis=1)
    
    # filter by home matches only
    concatenated_df = concatenated_df[concatenated_df['Venue'] == 'Home'].reset_index(drop=True)
    
    # drop 'Comp' and 'Venue' column
    concatenated_df.drop(['Comp', 'Venue', 'DaysLastPlayed'], axis=1, inplace = True)
        
    # rename features
    concatenated_df = concatenated_df.rename(columns={'xG': 'HxG', 'xGA': 'AxG', 'Poss': 'HPoss', 'Opponent': 'AwayTeam', 'team': 'HomeTeam'})
    
    if dataset2_df.empty:
        dataset2_df = concatenated_df
    else:
        dataset2_df = pd.concat([dataset2_df, concatenated_df]).reset_index(drop=True)

In [18]:
# Make sure we have 5 x 380 = 1900 matches in the DataFrame
dataset2_df.shape

(3800, 13)

In [19]:
dataset2_df.head()

,Date,Round,Day,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDays,ADays,HxG,AxG
0,2012-08-18,Matchweek 1,Sat,D,0,0,sunderland,NaN,arsenal,NaT,-274,NaN,NaN
1,2012-09-15,Matchweek 4,Sat,W,6,1,southampton,NaN,arsenal,13,13,NaN,NaN
2,2012-09-29,Matchweek 6,Sat,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,Matchweek 9,Sat,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,Matchweek 11,Sat,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


### 2.2.2 Converting Data Types

In [20]:
dataset2_df.dtypes

Date        datetime64[ns]
Round               object
Day                 object
Result              object
GF                  object
GA                  object
AwayTeam            object
HPoss              float64
HomeTeam            object
HDays               object
ADays               object
HxG                float64
AxG                float64
dtype: object

In [21]:
dataset2_df['GF'] = pd.to_numeric(dataset2_df['GF'])
dataset2_df['GA'] = pd.to_numeric(dataset2_df['GA'])
dataset2_df.head()

,Date,Round,Day,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDays,ADays,HxG,AxG
0,2012-08-18,Matchweek 1,Sat,D,0,0,sunderland,NaN,arsenal,NaT,-274,NaN,NaN
1,2012-09-15,Matchweek 4,Sat,W,6,1,southampton,NaN,arsenal,13,13,NaN,NaN
2,2012-09-29,Matchweek 6,Sat,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,Matchweek 9,Sat,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,Matchweek 11,Sat,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


### 2.2.3 Checking for Missing Values

In [22]:
dataset2_df.isnull().sum()

Date           0
Round          0
Day            0
Result         0
GF             0
GA             0
AwayTeam       0
HPoss        760
HomeTeam       0
HDays          0
ADays          0
HxG         1900
AxG         1900
dtype: int64

### 2.2.4 Data Transformation

In [23]:
dataset2_df['HDays'].value_counts()

7       829
3       749
4       586
6       330
5       303
8       267
14      210
13      106
9        72
15       67
2        55
10       42
11       38
-281     22
16       21
-274     14
12        9
17        6
21        6
-267      5
105       5
-351      5
-280      5
NaT       5
18        4
-273      4
-286      3
-253      3
-350      3
101       2
-266      2
-279      2
-251      2
-285      1
-255      1
-246      1
-252      1
-372      1
104       1
106       1
100       1
-272      1
-265      1
-295      1
-294      1
22        1
-291      1
25        1
-287      1
-283      1
-282      1
Name: HDays, dtype: int64

In [24]:
dataset2_df['ADays'].value_counts()

7       838
3       727
4       569
6       348
5       300
8       282
14      211
13      102
9        64
15       61
2        56
10       48
11       36
16       30
-281     22
-274     16
12       13
-280      7
-267      7
18        5
-273      4
21        4
19        3
-279      3
-253      3
102       3
-351      3
17        3
NaT       2
-251      2
105       2
-287      2
22        2
-275      2
-266      2
20        2
-352      1
-265      1
104       1
-371      1
-357      1
99        1
107       1
103       1
100       1
-284      1
23        1
-272      1
-258      1
-252      1
-250      1
-350      1
Name: ADays, dtype: int64

In [25]:
def transform_DaysLastPlayed(word):
    if word == 'NaT':
        return 10
    
    if int(word) < 0 or int(word) >= 10:
        return 10
    else:
        return int(word)

dataset2_df['HDays'] = dataset2_df['HDays'].apply(lambda x: transform_DaysLastPlayed(x))
dataset2_df['ADays'] = dataset2_df['ADays'].apply(lambda x: transform_DaysLastPlayed(x))
dataset2_df.head()

,Date,Round,Day,Result,GF,GA,AwayTeam,HPoss,HomeTeam,HDays,ADays,HxG,AxG
0,2012-08-18,Matchweek 1,Sat,D,0,0,sunderland,NaN,arsenal,10,10,NaN,NaN
1,2012-09-15,Matchweek 4,Sat,W,6,1,southampton,NaN,arsenal,10,10,NaN,NaN
2,2012-09-29,Matchweek 6,Sat,L,1,2,chelsea,NaN,arsenal,6,7,NaN,NaN
3,2012-10-27,Matchweek 9,Sat,W,1,0,qpr,NaN,arsenal,3,6,NaN,NaN
4,2012-11-10,Matchweek 11,Sat,D,3,3,fulham,NaN,arsenal,4,7,NaN,NaN


## 2.3 Dataset 3

In [26]:
dataset3_df = pd.read_csv(f'../data/raw/data-source-3/team-ratings.csv')

# remove irrelevant data
teams = list(rename_teams.keys())
dataset3_df = dataset3_df[dataset3_df['Team'].isin(teams)]

# rename team names in the 'Team' column for standardized team names
dataset3_df['Team'] = dataset3_df['Team'].apply(lambda word : rename_teams[word])

dataset3_df.head()

,Version,Date,Team,Attack,Midfield,Defense
0,FIFA 22,"Aug. 18, 2022",liverpool,86,84,85
1,FIFA 22,"Aug. 18, 2022",city,84,87,86
2,FIFA 22,"Aug. 18, 2022",chelsea,84,85,83
3,FIFA 22,"Aug. 18, 2022",united,82,83,81
4,FIFA 22,"Aug. 18, 2022",tottenham,83,81,78


In [27]:
dataset3_df.shape

(28406, 6)

### 2.3.2 Checking for Missing Values

In [28]:
dataset3_df.isnull().sum()

Version     0
Date        0
Team        0
Attack      0
Midfield    0
Defense     0
dtype: int64

### 2.3.3. Converting Data Types 

In [29]:
dataset3_df.dtypes

Version     object
Date        object
Team        object
Attack       int64
Midfield     int64
Defense      int64
dtype: object

In [30]:
def refactor_date(date):
    months_dict = {'Jan': '01', 'Feb': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
                   'July': '07', 'Aug': '08', 'Sept': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
    
    date = date.replace(',', '')
    date = date.replace('.', '')
    
    # replace first word of month 
    date = date.replace(date.split()[0], months_dict[date.split()[0]])
    date = date.replace(' ', '/')
    
    return date

In [31]:
dataset3_df['Date'] = dataset3_df['Date'].apply(lambda date: refactor_date(date))
dataset3_df['Date'] = pd.to_datetime(dataset3_df['Date'], format="%m/%d/%Y")
dataset3_df.head()

,Version,Date,Team,Attack,Midfield,Defense
0,FIFA 22,2022-08-18,liverpool,86,84,85
1,FIFA 22,2022-08-18,city,84,87,86
2,FIFA 22,2022-08-18,chelsea,84,85,83
3,FIFA 22,2022-08-18,united,82,83,81
4,FIFA 22,2022-08-18,tottenham,83,81,78


In [32]:
dataset3_df.to_csv(f'../data/processed/data-source-3/ratings-data-source-3.csv', index=False)

# 3. Integration of Datasets

## 3.1 Integrating Dataset 1 and Dataset 2

In [33]:
df

""


In [34]:
# merge two data sources into one DataFrame

df = dataset1_df
df = pd.merge(df, dataset2_df, on=['Date', 'HomeTeam', 'AwayTeam'])

In [35]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Round,Day,Result,GF,GA,HPoss,HDays,ADays,HxG,AxG
0,2012-08-18,arsenal,sunderland,0,0,D,0,0,D,C Foy,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.50,8.50,Matchweek 1,Sat,D,0,0,NaN,10,10,NaN,NaN
1,2012-08-18,fulham,norwich,5,0,W,2,0,H,M Oliver,11,4,9,2,12,11,6,3,0,0,0,0,1.80,3.60,4.50,Matchweek 1,Sat,W,5,0,NaN,10,10,NaN,NaN
2,2012-08-18,newcastle,tottenham,2,1,W,0,0,D,M Atkinson,6,12,4,6,12,8,3,5,2,2,0,0,2.50,3.40,2.75,Matchweek 1,Sat,W,2,1,NaN,10,10,NaN,NaN
3,2012-08-18,qpr,swansea,0,5,L,0,1,A,L Probert,20,12,11,8,11,14,5,3,2,2,0,0,2.00,3.40,3.80,Matchweek 1,Sat,L,0,5,NaN,10,10,NaN,NaN
4,2012-08-18,reading,stoke,1,1,D,0,1,A,K Friend,9,6,3,3,9,14,4,3,2,4,0,1,2.38,3.25,3.10,Matchweek 1,Sat,D,1,1,NaN,10,10,NaN,NaN


In [36]:
df.shape

(3800, 35)

## 3.2 Dropping Redundant Columns

In [37]:
df.drop(['FTR', 'GF', 'GA'], axis=1, inplace = True)
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,Round,Day,Result,HPoss,HDays,ADays,HxG,AxG
0,2012-08-18,arsenal,sunderland,0,0,0,0,D,C Foy,14,3,4,2,12,8,7,0,0,0,0,0,1.40,4.50,8.50,Matchweek 1,Sat,D,NaN,10,10,NaN,NaN
1,2012-08-18,fulham,norwich,5,0,2,0,H,M Oliver,11,4,9,2,12,11,6,3,0,0,0,0,1.80,3.60,4.50,Matchweek 1,Sat,W,NaN,10,10,NaN,NaN
2,2012-08-18,newcastle,tottenham,2,1,0,0,D,M Atkinson,6,12,4,6,12,8,3,5,2,2,0,0,2.50,3.40,2.75,Matchweek 1,Sat,W,NaN,10,10,NaN,NaN
3,2012-08-18,qpr,swansea,0,5,0,1,A,L Probert,20,12,11,8,11,14,5,3,2,2,0,0,2.00,3.40,3.80,Matchweek 1,Sat,L,NaN,10,10,NaN,NaN
4,2012-08-18,reading,stoke,1,1,0,1,A,K Friend,9,6,3,3,9,14,4,3,2,4,0,1,2.38,3.25,3.10,Matchweek 1,Sat,D,NaN,10,10,NaN,NaN


In [38]:
df.to_csv('../data/processed/integrated_dataset.csv', index=False)